In [92]:



from sqlalchemy import create_engine, Column, Integer, String, Boolean, Float, ForeignKey
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import json

Base = declarative_base()

class PlayerStats(Base):
    __tablename__ = 'player_stats'
    id = Column(Integer, primary_key=True)
    characterId = Column(String)
    competitiveTier = Column(String)
    partyId = Column(String)
    playerCard = Column(String)
    playerTitle = Column(String)
    puuid = Column(String)
    assists = Column(Integer)
    deaths = Column(Integer)
    kills = Column(Integer)
    playtimeMillis = Column(Integer)
    roundsPlayed = Column(Integer)
    score = Column(Integer)
    teamId = Column(String)
    match_id = Column(Integer, ForeignKey('match_info.id'))
    match = relationship('MatchInfo', back_populates='players')
    

class MatchInfo(Base):
    __tablename__ = 'match_info'
    id = Column(Integer, primary_key=True)
    customGameName = Column(String)
    gameLengthMillis = Column(Integer)
    gameMode = Column(String)
    gameStartMillis = Column(Integer)
    isCompleted = Column(Boolean)
    isRanked = Column(Boolean)
    mapId = Column(String)
    matchId = Column(String)
    provisioningFlowId = Column(String)
    queueId = Column(String)
    seasonId = Column(String)
    players = relationship('PlayerStats', back_populates='match')
    round_results = relationship('RoundResult', back_populates='match')
    teams = relationship('TeamStats', back_populates='match')

class RoundResult(Base):
    __tablename__ = 'round_results'
    id = Column(Integer, primary_key=True)
    match_id = Column(Integer, ForeignKey('match_info.id'))
    defuseLocationX = Column(Float)
    defuseLocationY = Column(Float)
    defuseRoundTime = Column(Integer)
    plantLocationX = Column(Float)
    plantLocationY = Column(Float)
    plantRoundTime = Column(Integer)
    plantSite = Column(String)
    roundCeremony = Column(String)
    roundNum = Column(Integer)
    roundResult = Column(String)
    roundResultCode = Column(String)
    winningTeam = Column(String)
    bombDefuser = Column(String)
    match = relationship('MatchInfo', back_populates='round_results')
    plant_player_locations = relationship('PlantPlayerLocation', back_populates='round_result')
    player_kills = relationship('PlayerKills', back_populates='round_result')

class PlantPlayerLocation(Base):
    __tablename__ = 'plant_player_locations'
    id = Column(Integer, primary_key=True)
    round_result_id = Column(Integer, ForeignKey('round_results.id'))
    locationX = Column(Float)
    locationY = Column(Float)
    puuid = Column(String)
    viewRadians = Column(Float)
    round_result = relationship('RoundResult', back_populates='plant_player_locations')

class PlayerKills(Base):
    __tablename__ = 'player_kills'
    id = Column(Integer, primary_key=True)
    round_result_id = Column(Integer, ForeignKey('round_results.id'))
    killer = Column(String)
    victim = Column(String)
    round_time = Column(Integer)
    player_locationX = Column(Float)
    player_locationY = Column(Float)
    victim_locationX = Column(Float)
    victim_locationY = Column(Float)
    killer_stats = Column(Integer, ForeignKey('player_stats.id'))
    victim_stats = Column(Integer, ForeignKey('player_stats.id'))
    round_result = relationship('RoundResult', back_populates='player_kills')
    killer_stats_rel = relationship('PlayerStats', foreign_keys=[killer_stats])
    victim_stats_rel = relationship('PlayerStats', foreign_keys=[victim_stats])

class TeamStats(Base):
    __tablename__ = 'team_stats'
    id = Column(Integer, primary_key=True)
    match_id = Column(Integer, ForeignKey('match_info.id'))
    numPoints = Column(Integer)
    roundsPlayed = Column(Integer)
    roundsWon = Column(Integer)
    teamId = Column(String)
    won = Column(Boolean)
    match = relationship('MatchInfo', back_populates='teams')

class Agents(Base):
    __tablename__ = 'agents'
    id = Column(Integer, primary_key=True)
    uuid = Column(String)
    agent = Column(String)

In [93]:
# Replace 'sqlite:///my_database.db' with the path to your SQLite database file.
engine = create_engine('sqlite:///Valorant.sqlite')

# Create the tables
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

In [94]:
from sqlalchemy_schemadisplay import create_schema_graph
from sqlalchemy import MetaData

graph = create_schema_graph(metadata=MetaData('sqlite:///Valorant.sqlite'))
graph.write_png('ER.png')

In [95]:
try:
    with open('../json/valorantmatches.json', 'r') as json_file:
        data = json.load(json_file)
except json.JSONDecodeError as e:
    print(f"JSON decoding error: {e}")
except FileNotFoundError:
    print("File not found.")
except Exception as e:
    print(f"An error occurred: {e}")
else:
    print("match JSON data loaded successfully.")

try:
    with open('../json/valorantcharacters.json', 'r') as char_json_file:
        char_data = json.load(char_json_file)
except json.JSONDecodeError as e:
    print(f"JSON decoding error: {e}")
except FileNotFoundError:
    print("File not found.")
except Exception as e:
    print(f"An error occurred: {e}")
else:
    print("character JSON data loaded successfully.")

match JSON data loaded successfully.
character JSON data loaded successfully.


In [100]:
pprint(char_data)

{'data': [{'abilities': [{'description': 'EQUIP Wingman. FIRE to send Wingman '
                                         'forward seeking enemies. Wingman '
                                         'unleashes a concussive blast toward '
                                         'the first enemy he sees. ALT FIRE '
                                         'when targeting a Spike site or '
                                         'planted Spike to have Wingman defuse '
                                         'or plant the Spike. To plant, Gekko '
                                         'must have the Spike in his '
                                         'inventory. When Wingman expires he '
                                         'reverts into a dormant globule. '
                                         'INTERACT to reclaim the globule and '
                                         'gain another Wingman charge after a '
                                         'short cooldown.',
   

In [96]:
char_data['data'][0]['uuid']

'e370fa57-4757-3604-3648-499e1f642d3f'

In [102]:
Session = sessionmaker(bind=engine)
session = Session()
for char in char_data['data']:
    characters = Agents(
        uuid=char['uuid'],
        agent=char['displayName']
    )

for match in data.get('matches', []):
    match_id = match['matchInfo']['matchId']  # Extract the match ID from the current match

    for player in match.get('players', []):
        player_stats = PlayerStats(
            characterId=player.get('characterId'),
            competitiveTier=player.get('competitiveTier'),
            partyId=player.get('partyId'),
            playerCard=player.get('playerCard'),
            playerTitle=player.get('playerTitle'),
            puuid=player.get('puuid'),
            assists=player['stats'].get('assists'),
            deaths=player['stats'].get('deaths'),
            kills=player['stats'].get('kills'),
            playtimeMillis=player['stats'].get('playtimeMillis'),
            roundsPlayed=player['stats'].get('roundsPlayed'),
            score=player['stats'].get('score'),
            teamId=player.get('teamId'),
            match_id=match_id,  # Set the match ID for the player_stats
        )
        session.add(player_stats)
session.commit() 


In [103]:
for match_data in data.get('matches', []):
    match_info = MatchInfo(
        customGameName=match_data['matchInfo']['customGameName'],
        gameLengthMillis=match_data['matchInfo']['gameLengthMillis'],
        gameMode=match_data['matchInfo']['gameMode'],
        gameStartMillis=match_data['matchInfo']['gameStartMillis'],
        isCompleted=match_data['matchInfo']['isCompleted'],
        isRanked=match_data['matchInfo']['isRanked'],
        mapId=match_data['matchInfo']['mapId'],
        matchId=match_data['matchInfo']['matchId'],
        provisioningFlowId=match_data['matchInfo']['provisioningFlowId'],
        queueId=match_data['matchInfo']['queueId'],
        seasonId=match_data['matchInfo']['seasonId']
    )

    # Add match_info to the session and commit to the database
    session.add(match_info)
    session.commit()

# Commit the session to save changes to the database
session.commit()

In [104]:
# Insert data into the RoundResult table
for match_data in data.get('matches', []):
    for round_result_data in match_data.get('roundResults', []):
        round_result = RoundResult(
            match_id=match_data['matchInfo']['matchId'], 
            defuseLocationX=round_result_data['defuseLocation']['x'],
            defuseLocationY=round_result_data['defuseLocation']['y'],
            defuseRoundTime=round_result_data['defuseRoundTime'],
            plantLocationX=round_result_data['plantLocation']['x'],
            plantLocationY=round_result_data['plantLocation']['y'],
            plantRoundTime=round_result_data['plantRoundTime'],
            plantSite=round_result_data['plantSite'],
            roundCeremony=round_result_data['roundCeremony'],
            roundNum=round_result_data['roundNum'],
            roundResult=round_result_data['roundResult'],
            roundResultCode=round_result_data['roundResultCode'],
            winningTeam=round_result_data['winningTeam'],
            bombDefuser=round_result_data.get('bombDefuser', None)
        )

        # Add round_result to the session and commit to the database
        session.add(round_result)
        session.commit()

# Commit the session to save changes to the database
session.commit()

In [76]:
for match_data in data.get('matches', []):
    for round_result_data in match_data.get('roundResults', []):
        for plant_player_location_data in round_result_data.get('plantPlayerLocations', []):
            plant_player_location = PlantPlayerLocation(
                round_result_id=round_result.id, 
                locationX=plant_player_location_data['location']['x'],
                locationY=plant_player_location_data['location']['y'],
                puuid=plant_player_location_data['puuid'],
                viewRadians=plant_player_location_data['viewRadians']
            )
            
            # Add plant_player_location to the session and commit to the database
            session.add(plant_player_location)
            session.commit()
# Commit the changes
session.commit()

In [77]:
for match_data in data.get('matches', []):
    for round_result_data in match_data.get('roundResults', []):
        for player_kills_data in round_result_data.get('playerStats', []):
            for kill_data in player_kills_data.get('kills', []):
                # Check if 'playerLocations' is not empty
                player_locations = kill_data.get('playerLocations', [])
                
                # Use the first location if available, otherwise set to None
                if player_locations:
                    player_locationX = player_locations[0]['location']['x']
                    player_locationY = player_locations[0]['location']['y']
                else:
                    player_locationX = None
                    player_locationY = None
                
                player_kills = PlayerKills(
                    round_result_id=round_result.id,
                    killer=kill_data['killer'],
                    victim=kill_data['victim'],
                    round_time=kill_data['timeSinceGameStartMillis'],
                    player_locationX=player_locationX,
                    player_locationY=player_locationY,
                    victim_locationX=kill_data['victimLocation']['x'],
                    victim_locationY=kill_data['victimLocation']['y'],
                    
                )
                
                # Add player_kills to the session and commit to the database
                session.add(player_kills)
                session.commit()
# Commit the changes
session.commit()

In [78]:
for match_data in data.get('matches', []):
    for team_stats_data in match_data.get('teams', []):
        team_stats = TeamStats(
            match_id=match_data['matchInfo']['matchId'],
            numPoints=team_stats_data['numPoints'],
            roundsPlayed=team_stats_data['roundsPlayed'],
            roundsWon=team_stats_data['roundsWon'],
            teamId=team_stats_data['teamId'],
            won=team_stats_data['won']
        )
        
        # Add team_stats to the session and commit to the database
        session.add(team_stats)
        session.commit()
# Commit the changes
session.commit()

In [79]:


# Close the session
session.close()